In [20]:
import whisper
#  INSTALAR WHISPER CON EL SIGUIENTE COMANDO:
# pip install git+https://github.com/openai/whisper.git 

import os
import numpy as np
import librosa
import torch
from pydub import AudioSegment
from transformers import WhisperProcessor, WhisperForConditionalGeneration

    # Seleccionamos el modelo que queremos utilizar
model_name= ["openai/whisper-tiny", "openai/whisper-base", 
             "openai/whisper-small", "openai/whisper-medium", 
             "openai/whisper-large", "openai/whisper-large-v2"][3] 

    # Especificamos el archivo de audio que queremos utilizar
audio_path = "ytmp3free.cc_the-cure-friday-im-in-love-youtubemp3free.org.mp3"

In [21]:
def export_model(model_name: str):
    """
        Función que exporta el modelo y el procesador de Whisper a un directorio
        Args:
            model_name: str, nombre del modelo a exportar
    """
    # Cargar el procesador desde el modelo preentrenado especificado
    processor = WhisperProcessor.from_pretrained(model_name)
    # Cargar el modelo desde el modelo preentrenado especificado
    model = WhisperForConditionalGeneration.from_pretrained(model_name)
    # Guardar el modelo en el directorio especificado por model_name
    model.save_pretrained(model_name)
    # Guardar el procesador en el mismo directorio
    processor.save_pretrained(model_name)
    return model, processor

# Verificar si el directorio del modelo no existe
if not os.path.exists(model_name): 
    # Exportar el modelo y el procesador si el directorio no existe
    model, processor = export_model(model_name)

In [22]:
def load_model(model_name: str):
    """
        Función que carga el modelo y el procesador de Whisper desde un directorio
        Args:
            model_name: str, nombre del directorio donde se encuentra el modelo y el procesador
    """
    # Cargar el procesador desde el directorio especificado
    processor = WhisperProcessor.from_pretrained(model_name)
    # Cargar el modelo desde el directorio especificado
    model = WhisperForConditionalGeneration.from_pretrained(model_name)
    return model, processor

# Cargar el modelo y el procesador utilizando el nombre del modelo especificado
model, processor = load_model("./" + model_name)

In [23]:
def segment_audio(audio_path: str, segment_duration_ms: int = 30000):
    """
        Función generador que segmenta un audio en segmentos de duración segment_duration_ms y los exporta a archivos .wav
        Args:
            audio_path: str, ruta al audio a segmentar
            segment_duration_ms: int, duración de los segmentos en milisegundos
    """
    
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(audio_path)
    # Obtener la duración del audio en milisegundos
    duration_ms = len(audio)

    # Iterar sobre el audio en pasos de segment_duration_ms
    for start_ms in range(0, duration_ms, segment_duration_ms):
        # Calcular el final del segmento
        end_ms = min(start_ms + segment_duration_ms, duration_ms)
        # Extraer el segmento del audio
        segment = audio[start_ms:end_ms]
        # Definir el nombre del archivo del segmento
        segment_path = f"segment_{start_ms // segment_duration_ms}.wav"
        # Exportar el segmento a un archivo .wav
        segment.export(segment_path, format="wav")
        # Devolver la ruta del archivo del segmento
        yield segment_path

In [24]:
def transcribe_long_audio(model: WhisperForConditionalGeneration, processor: WhisperProcessor, audio_path: str):
    """
        Función que transcribe un audio largo dividiéndolo en segmentos de 30 segundos
        Args:
            model: WhisperForConditionalGeneration, modelo de Whisper
            processor: WhisperProcessor, procesador de Whisper
            audio_path: str, ruta al audio
    """
    
    # Determinar si se utilizará GPU o CPU
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    full_transcription = ""  # Variable para almacenar la transcripción completa

    # Segmentar el audio y transcribir cada segmento
    for segment_path in segment_audio(audio_path):
        # Cargar el segmento de audio
        audio_data, _ = librosa.load(segment_path, sr=16000)
        # Procesar el audio para obtener las características de entrada
        input_features = processor(audio_data, return_tensors="pt", sampling_rate=16000).input_features
        input_features = input_features.to(device)
        # Generar la transcripción del segmento
        generated_ids = model.generate(input_features)
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        # Agregar la transcripción del segmento a la transcripción completa
        full_transcription += transcription + "\n"
        # Eliminar el archivo de segmento de audio
        os.remove(segment_path)

    return full_transcription.strip()  # Devolver la transcripción completa sin espacios en blanco al inicio y al final




In [25]:
# Ejecutamos el código
if __name__ == "__main__":
    transcription = transcribe_long_audio(model, processor, audio_path)
    print("Transcripción completa:")
    print(transcription)
    

Transcripción completa:
Stand by everyone and...CUT!
 I don't care if Monday's blue Tuesday's gray and Wednesday too Thursday, I don't care about you It's Friday, I'm in love Monday you can fall apart Tuesday, Wednesday break my heart Oh Thursday doesn't even start It's Friday, I'm in love
 Saturday way Sunday always comes to me Friday never has it end I don't care if Monday's black Tuesday, Wednesday, heart attack Thursday never looking back It's Friday, I'm in love
 Tuesday Wednesday
 See your shoes
 It's Friday, I'm in love! I don't care if Monday's blue Tuesday's gray and Wednesday too Thursday, I don't care about you It's Friday, I'm in love Monday you can fall apart Tuesday, Wednesday break my heart Thursday doesn't even start It's Friday, I'm in love
 Oh
 you
